# ID2214/FID3214 Assignment 2 Group no. 4
### Project members: 
Lucas Larsson, lulars@kth.se
Mihaela Bakšić, baksic@kth.se

### Declaration:
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214/FID3214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All parts of the assignment starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of all parts of the assignment starting 
with number 2 below, then the assignment will receive 2 points (in total).

Note that you do not have to develop the code directly within the notebook
but may instead copy the comments and test cases to a more convenient development environment
and when everything works as expected, you may paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).

## Load NumPy, pandas and time

In [3]:
import numpy as np
import pandas as pd
import time

In [4]:
from platform import python_version

print(f"Python version: {python_version()}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

Python version: 3.10.6
NumPy version: 1.23.3
Pandas version: 1.4.4


## Reused functions from Assignment 1

In [63]:
# Copy and paste functions from Assignment 1 here that you need for this assignment


# column filter

def create_column_filter(df):
    filter = []
    df_filtered = df.copy()
    for (col_name, col_data) in df_filtered.iteritems():
        if col_name in ['CLASS', 'ID'] or not df_filtered[col_name].dropna().nunique() <= 1:
            filter.append(col_name)

    return df_filtered[filter], filter

def apply_column_filter(df, filter):
    df_filtered = df.copy()[filter]
    return df_filtered


# Normalization

def create_normalization(df, normalizationtype='minimax'):
    assert normalizationtype in ['minmax', 'zscore']

    df_norm = df.copy()
    normalization = dict()

    for (col_name, col_data) in df_norm.select_dtypes(["int", 'float']).iteritems():
            if col_name not in ['CLASS', 'ID']:

                if normalizationtype == 'minmax': #minmax
                    min = df_norm[col_name].min()
                    max = df_norm[col_name].max()
                    df_norm[col_name] = [(x-min)/(max-min) for x in df_norm[col_name]]

                    normalization[col_name] = ('minmax', min, max)
                else: #zscore
                    mean = df_norm[col_name].mean()
                    std = df_norm[col_name].std()
                    df_norm[col_name] = df[col_name].apply(lambda x: (x - mean)/std)

                    normalization[col_name] = ('zscore', mean, std)


    return df_norm, normalization



def apply_normalization(df, normalization):

    df_norm = df.copy()

    for col_name in normalization:
        if normalization[col_name][0] == 'minmax': # minmax
            df_norm[col_name] = [(x-normalization[col_name][1])/(normalization[col_name][2]-normalization[col_name][1]) for x in df_norm[col_name]]
            df_norm[col_name] = df_norm[col_name].apply(lambda x: 0 if x<0 else (1 if x > 1 else x ) )

        else: # zscore
            df_norm[col_name] = df_norm[col_name].apply(lambda x: (x - normalization[col_name][1]) / normalization[col_name][2])

    return df_norm


# Imputation

def create_imputation(df):
    df_imp = df.copy()
    imputation = dict()

    for (col_name, col_data) in df_imp.select_dtypes(["int", 'float']).iteritems():
        if col_name not in ['CLASS', 'ID']:
            mean_col = df_imp[col_name].mean() if df_imp[col_name].count() != 0 else 0
            df_imp[col_name].fillna(mean_col, inplace=True)

            imputation[col_name] = mean_col

    for (col_name, col_data) in df_imp.select_dtypes(["object", 'category']).iteritems():
        if col_name not in ['CLASS', 'ID']:
            mode_col = df_imp[col_name].mode()[0]
            df_imp[col_name].fillna(mode_col, inplace=True)

            imputation[col_name] = mode_col

    return df_imp, imputation



def apply_imputation(df, imputation):
    df_imp = df.copy()

    for col_name in imputation:
        df_imp.fillna(imputation[col_name], inplace=True)

    return df_imp

# One Hot


def create_one_hot(df):
    copied_df = df.copy()
    second_copied_df = df.copy()
    one_hot_mapping = dict()

    excluded_columns = ['ID', 'CLASS']
    columns = list(df.select_dtypes(include=['object','category']))
    columns = [i for i in columns if i not in excluded_columns]

    for col in columns:
        copied_df[col] = copied_df[col].astype('category')
        one_hot_mapping[col] = list(copied_df[col].cat.categories)

        for i in one_hot_mapping[col]:
            col_name = col +'_'+ i
            new_col = copied_df[col].apply(lambda x: 1.0 if x == i else 0.0)
            second_copied_df[col_name] = new_col

        second_copied_df = second_copied_df.drop(columns = col, axis = 1)

    return second_copied_df, one_hot_mapping


def apply_one_hot(df, mapping):
    copied_df = df.copy()
    second_copied_df = df.copy()

    for col in copied_df.columns:
        if col in mapping.keys():
            for i in mapping[col]:
                col_name = col +'_'+ i
                new_col = copied_df[col].apply(lambda x: 1.0 if x == i else 0.0)
                second_copied_df[col_name] = new_col

            second_copied_df = second_copied_df.drop(columns = col, axis = 1)

    return second_copied_df


## AUC


def get_roc(predictions, correct_labels, score, index):

    crct = np.array(correct_labels)==predictions.columns[index]
    not_crct = np.array(correct_labels)!=predictions.columns[index]

    pred = predictions[predictions.columns[index]]

    tp = sum(pred[crct] >= score) / sum(crct)
    fp = sum(pred[not_crct] >= score) / sum(not_crct)

    return tp, fp


def auc(predictions, correctlabels):
    scores = np.unique(predictions)[::-1]
    nmbr_tot_labels = len(correctlabels)

    AUCs = {}

    for (idx,c) in enumerate(np.unique(correctlabels)):
        roc_pointes = [get_roc(predictions, correctlabels, score, idx) for score in scores]

        AUC = 0
        prev_tp = 0
        prev_fp = 0

        # we kept having problem with calculating AUC using the algorithm provided in the lecture slides.
        # So we just calculate the triangle and rectangles using the respective method to calculate
        # - tringle area = (height  * base) / 2
        # - rectangle area = len * wid
        for point in roc_pointes:
            tp, fp = point

            if tp > prev_tp and fp > prev_fp:
                AUC += (fp-prev_fp)*prev_tp + (fp-prev_fp)*(tp-prev_tp) / 2

            elif fp > prev_fp:
                AUC += (fp-prev_fp)*tp

            prev_tp = tp
            prev_fp = fp

        AUCs[c] = AUC

    AUC_tot = 0
    for col, auc in AUCs.items():
        AUC_tot +=  np.sum(np.array(correctlabels) == col) / nmbr_tot_labels * auc

    return AUC_tot


# Brier Score
def brier_score(dataframe, correctlabels):

    df = dataframe.copy()

    cor_df = pd.get_dummies(correctlabels)
    brierscore = np.mean(np.sum((df - cor_df)**2, axis=1))

    return brierscore

# Accuracy
def accuracy(dataframe, correctlabels):

    df = dataframe.copy()

    labels_df = df.idxmax(axis=1)
    truelabels_df = (labels_df == correctlabels).sum(axis=0)
    accuracy_df = truelabels_df/len(df)

    return accuracy_df



## 1. Define the class kNN

In [64]:
# Define the class kNN with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, imputation, normalization, one_hot, labels, training_labels, training_data, training_time
#
# Input to fit:
# self              - the object itself
# df                - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# normalizationtype - "minmax" (default) or "zscore"
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter   - a column filter (see Assignment 1) from df
# self.imputation      - an imputation mapping (see Assignment 1) from df
# self.normalization   - a normalization mapping (see Assignment 1), using normalizationtype from the imputed df
# self.one_hot         - a one-hot mapping (see Assignment 1)
# self.training_labels - a pandas series corresponding to the "CLASS" column, set to be of type "category" 
# self.labels          - a list of the categories (class labels) of the previous series
# self.training_data   - the values (an ndarray) of the transformed dataframe, i.e., after employing imputation, 
#                        normalization, and possibly one-hot encoding, and also after removing the "CLASS" and "ID" columns
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# k    - an integer >= 1 (default = 5)
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the relative class frequencies in the set of class labels from the k nearest 
#               (with respect to Euclidean distance) neighbors in training_data
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply column filtering, imputation, normalization and one-hot
#
# Hint 2: Get the numerical values (as an ndarray) from the resulting dataframe and iterate over the rows 
#         calling some sub-function, e.g., get_nearest_neighbor_predictions(x_test,k), which for a test row
#         (numerical input feature values) finds the k nearest neighbors and calculate the class probabilities.
#
# Hint 3: This sub-function may first find the distances to all training instances, e.g., pairs consisting of
#         training instance index and distance, and then sort them according to distance, and then (using the indexes
#         of the k closest instances) find the corresponding labels and calculate the relative class frequencies

class kNN():
    def __init__(self):
        self.column_filter = None
        self.imputation = None
        self.normalization = None
        self.one_hot = None
        self.labels = None
        self.training_labels = None
        self.training_data = None
        self.training_time = None

    def fit(self, df, normalizationtype = 'minmax'):
        filter_df, self.column_filter = create_column_filter(df)
        imp_df, self.imputation = create_imputation(filter_df)
        norm_df, self.normalization = create_normalization(imp_df, normalizationtype)
        one_hot_df, self.one_hot = create_one_hot(norm_df)
        self.training_labels = one_hot_df["CLASS"].astype('category')
        self.labels = np.unique(self.training_labels)
        self.training_data = np.array(one_hot_df.drop(['CLASS', 'ID'], axis=1))


    # I got help from a colleague data scientist at work on get_NN()
    def get_NN(self, test_x, k):

        length = len(self.training_data)

        distances = np.empty(length)

        for (index, train_x) in enumerate(self.training_data):
            distances[index] = distance(train_x, test_x)

        distances = np.column_stack((range(0, length), distances))
        sorted_indicies = distances[:,1].argsort()

        distances = distances[sorted_indicies]
        k_indicies = distances[:k, 0]
        k_labels = self.training_labels[k_indicies]
        unique, counts = np.unique(k_labels, return_counts=True)
        probabilities = np.column_stack((unique, counts / np.sum(counts)))
        return probabilities

    def predict(self, df, k=5):
        filter_df = apply_column_filter(df, self.column_filter)
        imp_df = apply_imputation(filter_df, self.imputation)
        norm_df = apply_normalization(imp_df, self.normalization)
        one_hot_df = apply_one_hot(norm_df, self.one_hot)

        # columns 'CLASS' and 'ID' are dropped after because some methods do not have adequate error handling
        one_hot_df = one_hot_df.drop(['CLASS', 'ID'], axis=1)

        number_columns = np.array(one_hot_df.select_dtypes(include=["int", 'float']))

        preds = pd.DataFrame(0.0, index=range(0,len(df)), columns = self.labels)

        for (index, row) in enumerate(number_columns):
            probs = self.get_NN(row, k)
            for prob in probs:
                preds.at[index, int(prob[0])] = prob[1]

        return preds


# euclidean distance using linear algebra library in numpy
def distance(x1, x2):
     return np.linalg.norm(x1-x2)


In [65]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

knn_model = kNN()

t0 = time.perf_counter()
knn_model.fit(glass_train_df)
print("Training time: {0:.2f} s.".format(time.perf_counter()-t0))

test_labels = glass_test_df["CLASS"]

k_values = [1,3,5,7,9]
results = np.empty((len(k_values),3))

for i in range(len(k_values)):
    t0 = time.perf_counter()
    predictions = knn_model.predict(glass_test_df,k=k_values[i])
    print("Testing time (k={0}): {1:.2f} s.".format(k_values[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=k_values,columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time: 0.01 s.
Testing time (k=1): 0.14 s.
Testing time (k=3): 0.15 s.
Testing time (k=5): 0.15 s.
Testing time (k=7): 0.15 s.
Testing time (k=9): 0.15 s.



'results'

,Accuracy,Brier score,AUC
1,0.747664,0.504673,0.810350
3,0.663551,0.488058,0.815859
5,0.579439,0.471028,0.833843
7,0.598131,0.471867,0.833481
9,0.616822,0.482981,0.827727


In [66]:
train_labels = glass_train_df["CLASS"]
predictions = knn_model.predict(glass_train_df,k=1)
print("Accuracy on training set (k=1): {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set (k=1): {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set (k=1): {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set (k=1): 1.0000
AUC on training set (k=1): 1.0000
Brier score on training set (k=1): 0.0000


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class NaiveBayes

In [6]:
# Define the class NaiveBayes with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self - the object itself
#
# Output from __init__:
# <nothing>
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# column_filter, binning, labels, class_priors, feature_class_value_counts, feature_class_counts
#
# Input to fit:
# self    - the object itself
# df      - a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins  - no. of bins (default = 10)
# bintype - either "equal-width" (default) or "equal-size" 
#
# Output from fit:
# <nothing>
#
# The result of applying this function should be:
#
# self.column_filter              - a column filter (see Assignment 1) from df
# self.binning                    - a discretization mapping (see Assignment 1) from df
# self.class_priors               - a mapping (dictionary) from the labels (categories) of the "CLASS" column of df,
#                                   to the relative frequencies of the labels
# self.labels                     - a list of the categories (class labels) of the "CLASS" column of df
# self.feature_class_value_counts - a mapping from the feature (column name) to the number of
#                                   training instances with a specific combination of (non-missing, categorical) 
#                                   value for the feature and class label
# self.feature_class_counts       - a mapping from the feature (column name) to the number of
#                                   training instances with a specific class label and some (non-missing, categorical) 
#                                   value for the feature
#
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Input to predict:
# self - the object itself
# df   - a dataframe
# 
# Output from predict:
# predictions - a dataframe with class labels as column names and the rows corresponding to
#               predictions with estimated class probabilities for each row in df, where the class probabilities
#               are estimated by the naive approximation of Bayes rule (see lecture slides)
#
# Hint 1: First apply the column filter and discretization
#
# Hint 2: Iterating over either columns or rows, and for each possible class label, calculate the relative
#         frequency of the observed feature value given the class (using feature_class_value_counts and 
#         feature_class_counts) 
#
# Hint 3: Calculate the non-normalized estimated class probabilities by multiplying the class priors to the
#         product of the relative frequencies
#
# Hint 4: Normalize the probabilities by dividing by the sum of the non-normalized probabilities; in case
#         this sum is zero, then set the probabilities to the class priors
#
# Hint 5: To clarify the assignment text a little: self.feature_class_value_counts should be a mapping from 
#         a column name (a specific feature) to another mapping, which given a class label and a value for 
#         the feature, returns the number of training instances which have included this combination, 
#         i.e., the number of training instances with both the specific class label and this value on the feature.
#
# Hint 6: As an additional hint, you may take a look at the slides from the NumPy and pandas lecture, to see how you 
#         may use "groupby" in combination with "size" to get the counts for combinations of values from two columns.




In [7]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.csv")

glass_test_df = pd.read_csv("glass_test.csv")

nb_model = NaiveBayes()

test_labels = glass_test_df["CLASS"]

nobins_values = [3,5,10]
bintype_values = ["equal-width","equal-size"]
parameters = [(nobins,bintype) for nobins in nobins_values for bintype in bintype_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    nb_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = nb_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values]),
                       columns=["Accuracy","Brier score","AUC"])

print()
display("results",results)

Training time (3, 'equal-width'): 0.10 s.
Testing time (3, 'equal-width'): 0.09 s.
Training time (3, 'equal-size'): 0.09 s.
Testing time (3, 'equal-size'): 0.07 s.
Training time (5, 'equal-width'): 0.10 s.
Testing time (5, 'equal-width'): 0.08 s.
Training time (5, 'equal-size'): 0.08 s.
Testing time (5, 'equal-size'): 0.07 s.
Training time (10, 'equal-width'): 0.09 s.
Testing time (10, 'equal-width'): 0.07 s.
Training time (10, 'equal-size'): 0.10 s.
Testing time (10, 'equal-size'): 0.07 s.



'results'

Accuracy  Brier score       AUC
3  equal-width  0.616822     0.622116  0.724335
   equal-size   0.607477     0.554782  0.780163
5  equal-width  0.644860     0.551101  0.771688
   equal-size   0.598131     0.581556  0.796675
10 equal-width  0.654206     0.527569  0.812887
   equal-size   0.588785     0.741668  0.751165

In [8]:
train_labels = glass_train_df["CLASS"]
nb_model.fit(glass_train_df)
predictions = nb_model.predict(glass_train_df)
print("Accuracy on training set: {0:.4f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.4f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.4f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.8505
AUC on training set: 0.9687
Brier score on training set: 0.2263


### Comment on assumptions, things that do not work properly, etc.